# 毎月・毎週の頭に実行

In [ ]:
#@title GAレポート更新 →_ch-m
# GA APIでデータを抽出して集計した結果をGoogle Sheetsへ保存する
# sim@bで実行すること

#@markdown 書き込むGoogle Sheets
# URL = "https://docs.google.com/spreadsheets/d/1wz20DITKF1GpIryhifiPShEbSgIVwSqlD22WEL6dpCY" #@param {type:"string"}
URL = "https://docs.google.com/spreadsheets/d/10QO3zJeHQKLpBMhv-93PTHVtRIQ7UlLul3vuXOXO58k"

try:
    from google.colab import data_table, files as colab_files
    IN_COLAB = True
except Exception:
    data_table = None
    colab_files = None
    IN_COLAB = False

try:
    from megaton import files, start
except ModuleNotFoundError:
    if IN_COLAB:
        %pip install -U -q git+https://github.com/mak00s/megaton
        from megaton import files, start
    else:
        _pip_install(["install", "-e", "."])
        from megaton import files, start  # noqa: F401

if IN_COLAB and colab_files is not None:
    files = colab_files

import numpy as np
import pandas as pd
import pytz
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.display import clear_output
from urllib.parse import urlparse

TIMEZONE = "Asia/Tokyo"
KEY_COLS = ['month', 'clinic', 'channel', 'medium', 'source', 'users', 'cv', 'ad_cost']

def get_past_date(n_days=None, n_months=None, timezone=TIMEZONE, return_date_obj=False):
    """
    Returns a date N days ago, or the 1st of N months ago. Defaults to today.
    """
    now = datetime.now(pytz.timezone(timezone))

    if n_days is None and n_months is None:
        result_date = now
    elif n_days is not None and n_months is not None:
        raise ValueError("Specify either 'n_days' or 'n_months', not both.")
    elif n_days is not None:
        result_date = now - timedelta(days=n_days)
    else:
        result_date = now.replace(day=1) - relativedelta(months=n_months)

    return result_date.date() if return_date_obj else result_date.strftime("%Y-%m-%d")

def parse_end_date(raw_date_str):
    """
    Parse a date string in various formats:
    YYYY-MM-DD, YYYYMMDD, YYYY-MM, YYYYMM → datetime
    If only year+month is provided, returns last day of that month.
    """
    raw_date_str = raw_date_str.strip().replace("/", "-")

    try:
        if re.fullmatch(r"\d{8}", raw_date_str):  # YYYYMMDD
            return datetime.strptime(raw_date_str, "%Y%m%d")
        elif re.fullmatch(r"\d{6}", raw_date_str):  # YYYYMM
            dt = datetime.strptime(raw_date_str, "%Y%m")
            return (dt.replace(day=1) + relativedelta(months=1)) - timedelta(days=1)
        elif re.fullmatch(r"\d{4}-\d{2}-\d{2}", raw_date_str):  # YYYY-MM-DD
            return datetime.strptime(raw_date_str, "%Y-%m-%d")
        elif re.fullmatch(r"\d{4}-\d{2}", raw_date_str):  # YYYY-MM
            dt = datetime.strptime(raw_date_str, "%Y-%m")
            return (dt.replace(day=1) + relativedelta(months=1)) - timedelta(days=1)
        else:
            raise ValueError("Invalid date format")
    except Exception as e:
        raise ValueError(f"❌ end_dateの形式が不正です: {e}")

def get_report_range(target_months_ago):
    now = datetime.now(pytz.timezone(TIMEZONE))

    if target_months_ago == 0:
        # date_from = 13ヶ月前の月初
        base = now.replace(day=1)
        date_from = (base - relativedelta(months=12)).date().isoformat()
        # date_to = 昨日
        date_to = (now - timedelta(days=1)).date().isoformat()
    else:
        base = now.replace(day=1) - relativedelta(months=target_months_ago)
        date_from = (base - relativedelta(months=12)).date().isoformat()
        date_to = (base + relativedelta(months=1) - timedelta(days=1)).date().isoformat()

    return date_from, date_to

def apply_normalization(df, column, pattern_map, new_col=None, drop_original=False):
    """
    Normalizes a column using regex-based pattern matching and custom actions.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The column to normalize.
        pattern_map (dict): Mapping of regex patterns to replacements or actions.
        new_col (str, optional): Column name to store result in (default: overwrite original).
        drop_original (bool): If True and new_col is set, drop the original column.

    Returns:
        pd.DataFrame
    """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")

    def normalize(value):
        if not isinstance(value, str):
            return value
        value = value.lower()
        for pattern, action in pattern_map.items():
            try:
                if re.search(pattern, value):
                    if action == 'remove_m_prefix':
                        return re.sub(r'^m\.', '', value)
                    elif action == 'remove_www_prefix':
                        return re.sub(r'^www\.', '', value)
                    # Add more custom actions here if needed
                    else:
                        return action  # regular substitution
            except re.error as err:
                print(f"⚠️ Skipping bad pattern '{pattern}': {err}")
        return value

    target_col = new_col or column
    df[target_col] = df[column].apply(normalize)

    if drop_original and new_col:
        df.drop(columns=[column], inplace=True)

    return df

def classify_channel(row):
    ch = row.get("channel", "")
    med = row.get("medium", "").lower()
    src = row.get("source", "").lower().replace("www.", "")

    # --- AIチャネル検出 ---
    ai_keywords = ["bard", "chatgpt", "claude", "copilot", "gemini", "perplexity"]
    if any(keyword in src for keyword in ai_keywords) or any(keyword in med for keyword in ai_keywords):
        return "AI"

    # --- Map 判定 ---
    if med == "map" or re.search(r'(^|\.)maps?\.', src):
        return "Map"

    # --- Referral 再判定 ---
    if ch == "Referral":
        organic_keywords = ["search", "docomo.ne.jp", ".jword.jp", "jp.hao123.com"]
        if any(keyword in src for keyword in organic_keywords):
            return "Organic Search"

        sns_keywords = ["threads.net", "threads"]
        if any(keyword in src for keyword in sns_keywords):
            return "Organic Social"

        if any(domain in src for domain in GROUP_DOMAINS):
            return "Group"

    # fallback: original GA4 channel
    return ch

def infer_clinic_from_lp(lp_val):
    if not isinstance(lp_val, str) or not lp_val:
        return "不明"

    # 1️⃣ dentamap_id で判定
    for site in sites:
        if "dentamap_id" in site and f"id={site['dentamap_id']}" in lp_val:
            return site["clinic"]

    # 2️⃣ id=数字 が含まれる場合は数字を返す
    match = re.search(r"id=(\d+)", lp_val)
    if match:
        return match.group(1)

    # 3️⃣ ドメインからクリニック名を推定
    parsed = urlparse(lp_val if "://" in lp_val else f"http://{lp_val}")
    domain = parsed.netloc.lower()

    clinic_map = {
        "sapporo": "札幌",
        "sendai": "仙台",
        "ikebukuro": "池袋",
        "shinjuku": "新宿",
        "shibuya": "渋谷",
        "tokyo": "東京",
        "yokohama": "横浜",
        "umeda": "梅田",
        "namba": "難波",
        "hakata": "博多",
        "tenjin": "天神",
    }

    for key, name in clinic_map.items():
        if key in domain:
            return name

    # 4️⃣ どれにも該当しない場合
    return "不明"

# Main report generation
def generate_channel_report(
    end_date=None,
    num_months=1,
    sheet_name="_ch-m",
    sheet_url=None,
    cell_sheet=None,
    cell_start=None,
    cell_end=None,
    clinic_filter=None
):
    """
    Generate GA4 channel report and optionally save to Google Sheets.

    Args:
        end_date (str, optional): Accepts YYYY-MM-DD, YYYYMMDD, YYYY-MM, or YYYYMM. Defaults to yesterday.
        num_months (int): Number of months to include.
        sheet_name (str): Sheet tab name for saving.
        sheet_url (str, optional): Google Sheet URL. If set, saves automatically.
        cell_sheet (str): Sheet to write start/end date into.
        cell_start (str): Cell for start date (e.g., "A1").
        cell_end (str): Cell for end date (e.g., "B1").
        clinic_filter (list[str], optional): List of clinic names to include (e.g., ["渋谷", "新宿"]).
    """
    if end_date is None:
        end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

    end_dt = parse_end_date(end_date)
    start_dt = end_dt.replace(day=1) - relativedelta(months=(num_months - 1))
    start_date = start_dt.strftime("%Y-%m-%d")
    end_date = end_dt.strftime("%Y-%m-%d")

    mg.report.set_dates(start_date, end_date)
    print(f"📅 レポート期間：{start_date}〜{end_date} ({TARGET_MONTHS_AGO})")

    dfs = []
    selected_sites = [s for s in sites if not clinic_filter or s["clinic"] in clinic_filter]
    # print(f"🔍 対象クリニック: {', '.join(s['clinic'] for s in selected_sites)}")

    # --- 各クリニックの処理 ---
    for site in selected_sites:
        clinic_name = site['clinic']
        print(f"🔄 {clinic_name}...", end='')
        ga4_property_id = site['ga4_property_id']
        mg.ga['4'].property.id = ga4_property_id

        # Step 1: CV & ad cost
        mg.report.run(d=[("yearMonth", "month"), ("defaultChannelGroup", "channel"), "medium", "source"],
                      m=[(site["cv"], "cv"), ("advertiserAdCost", "ad_cost")])
        df_metrics = mg.report.data

        # Step 2: Active users
        mg.report.run(d=[("yearMonth", "month"), ("sessionDefaultChannelGroup", "channel"),
                         ("sessionMedium", "medium"), ("sessionSource", "source")],
                      m=[("activeUsers", "users")])
        df_users = mg.report.data

        if df_users is not None and df_metrics is not None:
            merged_df = pd.merge(df_users, df_metrics, on=["month", "channel", "medium", "source"], how="left")
        else:
            merged_df = pd.DataFrame(columns=KEY_COLS)
            print(f"⚠️ データなし: {clinic_name}")

        merged_df['clinic'] = clinic_name
        dfs.append(merged_df)

    # --- dentamap 用のセッション & CV レポートを追加 ---
    clinic_name = "dentamap"
    print(f"🔄 {clinic_name}")
    site = next((s for s in selected_sites if s["clinic"] == clinic_name), None)
    mg.ga['4'].property.id = site["ga4_property_id"]
    mg.report.run(
        d=[
            ("yearMonth", "month"),
            ("sessionDefaultChannelGroup", "channel"),
            ("sessionSource", "source"),
            ("landingPagePlusQueryString", "lp")
        ],
        filter_d="sessionDefaultChannelGroup==Organic Social;landingPagePlusQueryString=@/apl/netuser/?id=",
        m=[
            ("activeUsers", "users"),
            ("keyEvents", "cv")
        ]
    )
    if mg.report.data is not None:
        df_dentamap = mg.report.data.copy()
        # def infer_clinic_from_lp(lp_val):
        #     for site in sites:
        #         if "dentamap_id" in site and f"id={site['dentamap_id']}" in lp_val:
        #             return site["clinic"]
        #     # clinic を特定できない場合は、id=XXXX の数字部分をそのまま使う
        #     match = re.search(r"id=(\d+)", lp_val)
        #     return match.group(1) if match else "不明"

        df_dentamap['clinic'] = df_dentamap['lp'].apply(infer_clinic_from_lp)
        df_dentamap['medium'] = "dentamap"
        # KEY_COLS に必要な列が足りない場合は補完
        for col in KEY_COLS:
            if col not in df_dentamap.columns:
                df_dentamap[col] = None
        dfs.append(df_dentamap[KEY_COLS])

    # 共通の前処理
    df_all = pd.concat([df for df in dfs if not df.empty], ignore_index=True)

    # source 正規化
    df_all = apply_normalization(df_all, 'source', source_map)

    # channel 再分類
    df_all['channel'] = df_all.apply(classify_channel, axis=1)

    # 数値変換（既存処理）
    df_all['cv'] = pd.to_numeric(df_all['cv'], errors='coerce').fillna(0).astype(int)
    df_all['ad_cost'] = pd.to_numeric(df_all['ad_cost'], errors='coerce').fillna(0).astype(int)

    # 列順を統一
    df_result = df_all[KEY_COLS]

    if sheet_url:
        if cell_sheet and cell_start and cell_end:
            mg.sheets_service.update_cells(sheet_url, cell_sheet, {cell_start: start_date, cell_end: end_date})
        mg.sheets_service.upsert_df(
            sheet_url,
            sheet_name,
            df_result,
            keys=["month", "clinic"],
            columns=KEY_COLS,
            sort_by=["month", "clinic", "channel", "medium", "source"],
            create_if_missing=True,
        )

    return df_result

def classify_page(df, mapping, new_col='page_category'):
    """
    Classifies pages based on regex patterns.

    Args:
        df (pd.DataFrame): DataFrame with a 'page' or 'landing_page' column.
        mapping (dict): {regex_pattern: category_label}
        new_col (str): Column name to write classification to.

    Returns:
        pd.DataFrame: Original df with an added classification column.
    """
    if df.empty or not mapping:
        return df

    def map_page(url):
        for pattern, label in mapping.items():
            try:
                if re.search(pattern, url):
                    return label
            except re.error as e:
                print(f"⚠️ Invalid regex pattern '{pattern}': {e}")
        return "other"  # fallback

    df = df.copy()
    df[new_col] = df['page'].apply(map_page)
    return df

# Setup
CREDS_PATH = "/nbs/key/sa-shibuya-kyousei.json"
mg = start.Megaton(CREDS_PATH)

GA4_ACCOUNT = '141366107'
GA4_PROPERTY = '254470346'
mg.ga['4'].account.select(GA4_ACCOUNT)
mg.ga['4'].property.select(GA4_PROPERTY)

# BQ準備
GCP_PROJECT = 'shibuya-kyousei'
bq = mg.launch_bigquery(GCP_PROJECT)
clear_output()
print(f"✅ BigQueryを使う準備ができました。")

# Run Report

# 対象クリニックとsource/pageのマッピングルールをGSから取得
source_map = {}
page_map = {}
sites = []
try:
    if mg.open.sheet(URL):
        mg.gs.sheet.select('config')
        sites = mg.gs.sheet.data or []
        df_config = pd.DataFrame(sites)
        if not {'clinic', 'min_impressions', 'max_position'}.issubset(df_config.columns):
            raise ValueError("⚠️ 'config' sheet must contain 'clinic', 'min_impressions', and 'max_position' columns.")
        df_config['min_impressions'] = pd.to_numeric(df_config['min_impressions'], errors='coerce').fillna(10)
        df_config['max_position'] = pd.to_numeric(df_config['max_position'], errors='coerce').fillna(50)

        mg.gs.sheet.select('source_map')
        df_map = pd.DataFrame(mg.gs.sheet.data)
        if {'pattern', 'normalized'}.issubset(df_map.columns):
            source_map = dict(zip(df_map['pattern'], df_map['normalized']))
        else:
            raise ValueError("⚠️ 'source_map' sheet must include 'pattern' and 'normalized' columns.")

        try:
            mg.gs.sheet.select('page_map')
            df_pmap = pd.DataFrame(mg.gs.sheet.data)
            if {'pattern', 'category'}.issubset(df_pmap.columns):
                page_map = dict(zip(df_pmap['pattern'], df_pmap['category']))
            else:
                raise ValueError("⚠️ 'page_map' must include 'pattern' and 'category' columns.")
        except Exception as e:
            print(f"⚠️ Failed to load page_map: {e}")

except PermissionError:
    print("❌ スプレッドシートのアクセス権がありません。該当のGoogle Sheetsにサービスアカウントの編集権限を付与してください。")
except Exception as e:
    print("❌ シートの書き込み中にエラーが発生しました:", e)

GROUP_DOMAINS = set(site['domain'].replace('www.', '') for site in sites if 'domain' in site)
GROUP_DOMAINS |= {
    'invisalignteen-kyousei.jp',
    'mienaikyousei.tokyo',
    'mienaiuragawakyousei.tokyo',
}

CLINIC_FILTER = [
    "札幌",
    "仙台",
    "池袋",
    "新宿",
    "渋谷",
    "東京",
    "横浜",
    "梅田",
    "難波",
    "博多",
    "天神",
    # "Gr.",
    "dentamap",
]

# Load query_map from sheet
query_map = {}
try:
    mg.gs.sheet.select('query_map')
    df_qmap = pd.DataFrame(mg.gs.sheet.data)
    if {'pattern', 'mapped_to'}.issubset(df_qmap.columns):
        query_map = dict(zip(df_qmap['pattern'], df_qmap['mapped_to']))
    else:
        raise ValueError("⚠️ 'query_map' must include 'pattern' and 'mapped_to' columns.")
except Exception as e:
    print(f"⚠️ Failed to load query_map: {e}")

# レポート期間
#@markdown レポート対象月（0 = 当月, 1 = 先月, 2 = 2ヶ月前 ...）
TARGET_MONTHS_AGO = 0  #@param {type: "number"}
date_from, date_to = get_report_range(TARGET_MONTHS_AGO)
# print(f"期間：{date_from}〜{date_to}")

df = generate_channel_report(
    end_date=date_to,
    # num_months=13,
    num_months=1,
    clinic_filter=CLINIC_FILTER,
    sheet_url=URL,
    sheet_name="_ch-m",
    cell_sheet="CV",
    cell_start="L1",
    cell_end="N1"
)
mg.show.table(df, rows=10)


In [ ]:
#@title GAからLPのCVを得て分類 →_lp_cat

# 同じ期間でGA4データを取得して保存
# date_from = get_past_date(n_months=TARGET_MONTHS_AGO)
# date_to = (get_past_date(n_months=TARGET_MONTHS_AGO - 1, return_date_obj=True) - timedelta(days=1)).isoformat()
print(f"期間：{date_from}〜{date_to}")

mg.report.set_dates(date_from, date_to)

dfs = []
selected_sites = [s for s in sites if not CLINIC_FILTER or s["clinic"] in CLINIC_FILTER]

for site in selected_sites:
    clinic = site['clinic']
    ga4_id = site['ga4_property_id']
    mg.ga['4'].property.id = ga4_id
    print(f"{clinic} ({ga4_id}) ", end='')

    # Get users
    mg.report.run(
        d=[("yearMonth", "month"), ("landingPage", "page")],
        m=[("activeUsers", "users")],
        filter_d="sessionDefaultChannelGroup==Organic Search"
    )
    df_users = mg.report.data

    # Get CVs
    mg.report.run(
        d=[("yearMonth", "month"), ("landingPage", "page")],
        m=[("totalPurchasers", "cv")],
        filter_d="defaultChannelGroup==Organic Search"
    )
    df_cv = mg.report.data if mg.report.data is not None else pd.DataFrame(columns=["month", "page", "cv"])

    # Merge & label
    df = pd.merge(df_users, df_cv, on=["month", "page"], how="outer").fillna(0)
    df['clinic'] = clinic
    df['users'] = df['users'].astype(int)
    df['cv'] = df['cv'].astype(int)

    dfs.append(df)

df_lp = pd.concat(dfs, ignore_index=True)

df_lp = classify_page(df_lp, page_map, new_col='page_category')

# mg.show.table(df_lp, rows=10)

# 集計して_lp_catシートに保存

landing_summary_df = (
    df_lp
    .groupby(['month', 'clinic', 'page_category'], as_index=False)
    .agg(
        users=('users', 'sum'),
        cv=('cv', 'sum')
    )
    .sort_values(['month', 'clinic', 'users'], ascending=[True, True, False])
)

mg.sheets_service.upsert_df(
    URL,
    "_lp_cat",
    landing_summary_df,
    keys=["month", "page_category"],
    columns=["month", "clinic", "page_category", "users", "cv"],
    sort_by=["month", "page_category"],
    create_if_missing=True,
)
mg.show.table(landing_summary_df, rows=10)


In [ ]:
#@title 3.GAからSNS流入データ（直接）を得て分類

# date_from = "2024-11-01"
# date_to = "2025-10-31"
# print(f"期間：{date_from}〜{date_to}")
# mg.report.set_dates(date_from, date_to)

dfs = []
selected_sites = [s for s in sites if s["clinic"] in CLINIC_FILTER]
# print(f"🔍 対象クリニック: {', '.join(s['clinic'] for s in selected_sites)}")
# --- 各クリニックの処理 ---
for site in selected_sites:
    clinic_name = site['clinic']
    if clinic_name == "dentamap":
        continue  # dentamapはスキップ
    print(f"🔄 {clinic_name}...", end='')
    ga4_property_id = site['ga4_property_id']
    mg.ga['4'].property.id = ga4_property_id

    mg.report.run(
        d=[
            ("yearMonth", "month"),
            ("landingPage", "lp"),
            ("sessionMedium", "medium"),
            ("sessionSource", "source"),
            ("sessionCampaignName", "campaign"),
            ("sessionManualAdContent", "content"),
        ],
        m=[("activeUsers", "users"), "sessions", ("totalPurchasers", "cv")]
    )
    merged_df = mg.report.data
    merged_df['clinic'] = clinic_name

    # 🔧 LP の前に site['url'] を付与（「//」防止）
    base_url = site['url'].rstrip('/')  # 末尾の / を削除
    merged_df['lp'] = base_url + merged_df['lp']

    dfs.append(merged_df)

# --- dentamap 用のセッション & CV レポートを追加 ---
clinic_name = "dentamap"
print(f"🔄 {clinic_name}")
site = [s for s in sites if s["clinic"] in [clinic_name]][0]
mg.ga['4'].property.id = site["ga4_property_id"]
mg.report.run(
    d=[
        ("yearMonth", "month"),
        ("landingPagePlusQueryString", "lp"),
        ("sessionMedium", "medium"),
        ("sessionSource", "source"),
        ("sessionCampaignName", "campaign"),
        ("sessionManualAdContent", "content"),
    ],
    filter_d="landingPagePlusQueryString=@/apl/netuser/?id=",
    m=[
        ("activeUsers", "users"),
        "sessions",
        ("keyEvents", "cv")
    ]
)
if mg.report.data is not None:
    df_dentamap = mg.report.data.copy()
    def infer_clinic_from_lp(lp_val):
        for site in sites:
            if "dentamap_id" in site and f"id={site['dentamap_id']}" in lp_val:
                return site["clinic"]
        # clinic を特定できない場合は、id=XXXX の数字部分をそのまま使う
        match = re.search(r"id=(\d+)", lp_val)
        return match.group(1) if match else "不明"

    df_dentamap['clinic'] = df_dentamap['lp'].apply(infer_clinic_from_lp)
    df_dentamap['lp'] = df_dentamap['lp'].str.extract(r'(/apl/netuser/\?id=\d+)')
    df_dentamap['lp'] = "https://plus.dentamap.jp" + df_dentamap['lp']

    # param分散をまとめる
    df_dentamap = (
        df_dentamap
        .groupby(["month", "lp", "medium", "source", "campaign", "content", "clinic"], as_index=False)
        .agg({
            "users": "sum",
            "sessions": "sum",
            "cv": "sum"
        })
    )
    # df_dentamap['medium'] = "dentamap"
    # KEY_COLS に必要な列が足りない場合は補完
    # for col in KEY_COLS:
    #     if col not in df_dentamap.columns:
    #         df_dentamap[col] = None
    # dfs.append(df_dentamap[KEY_COLS])

# 共通の前処理
df_all = pd.concat(
    [df for df in dfs if not df.empty] + [df_dentamap],
    ignore_index=True
)

# campaign 正規化
df_all["campaign"] = df_all["campaign"].str.replace(r"\([^)]*\)", "", regex=True)

# content 正規化
df_all["content"] = df_all["content"].str.replace(r"\([^)]*\)", "", regex=True)

# source 正規化
# df_all = apply_normalization(df_all, 'source', source_map)

# channel 再分類
# df_all['channel'] = df_all.apply(classify_channel, axis=1)

# 数値変換（既存処理）
df_all['cv'] = pd.to_numeric(df_all['cv'], errors='coerce').fillna(0).astype(int)

col_order = ["month", "clinic", "medium", "source", "campaign", "content", "lp", "users", "cv"]

df_filtered = df_all[
    (df_all["medium"] == "social") |
    (df_all["source"].str.contains(r'\b(tiktok|instagram|youtube|t\.co)\b', case=False, na=False))
][col_order]

# Google Sheets保存
sheet_url = "https://docs.google.com/spreadsheets/d/1nUoS71GW2ej570FYp8RHZvLwFv5CWwGgV8i2BzuslL0"
sheet_name = "_ga"
mg.sheets_service.upsert_df(
    sheet_url,
    sheet_name,
    df_filtered,
    keys=["month", "clinic", "lp", "source", "medium", "campaign", "content"],
    columns=["month", "clinic", "medium", "source", "campaign", "content", "lp", "users", "cv"],
    sort_by=["month", "clinic", "medium", "source", "campaign", "content", "lp", "users", "cv"],
    create_if_missing=True,
)


In [ ]:
#@title G広告・SNSからkyousei-clinic.jp / dentamapへの流入

table_from = date_from.replace('-', '')
table_from = "20251101"
table_to = date_to.replace('-', '')
print(f"期間：{table_from}〜{table_to}")

df_h = bq.run(query=f"""--Colab
WITH base AS (
  SELECT
    FORMAT_DATE('%Y%m', PARSE_DATE('%Y%m%d', event_date)) AS month,
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS ga_session_id,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'medium') AS medium,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'source') AS source,
    event_name,
    event_timestamp,

    -- session_start のみ page_location を抽出してlpを設定
    CASE
      WHEN event_name = 'session_start' THEN (
        SELECT
          COALESCE(
            REGEXP_EXTRACT(value.string_value, r'^[^#?]*\\?id=\\d+'),
            REGEXP_REPLACE(value.string_value, r'\\?.*', '')
          )
        FROM UNNEST(event_params)
        WHERE key = 'page_location'
      )
    END AS lp,

    CASE WHEN event_name LIKE 'dentamap_submit_%' THEN 1 ELSE 0 END AS cv
  FROM `shibuya-kyousei.analytics_492311970.events_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '{table_from}' AND '{table_to}'
    AND (event_name IN ('session_start', 'page_view') OR event_name LIKE 'dentamap_submit_%')
),
-- セッション内で埋める
enriched AS (
  SELECT
    b.*,
    LAST_VALUE(lp IGNORE NULLS) OVER (
      PARTITION BY user_pseudo_id, ga_session_id
      ORDER BY event_timestamp
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS session_lp,
    -- medium, source は最初の非NULLを使う
    LAST_VALUE(medium IGNORE NULLS) OVER (
      PARTITION BY user_pseudo_id, ga_session_id
      ORDER BY event_timestamp
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS session_medium,
    LAST_VALUE(source IGNORE NULLS) OVER (
      PARTITION BY user_pseudo_id, ga_session_id
      ORDER BY event_timestamp
      ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS session_source
  FROM base AS b
),
-- users（母数）とCVユーザーをそれぞれ抽出
users AS (
  SELECT DISTINCT
    month,
    user_pseudo_id,
    session_medium AS medium,
    session_source AS source,
    session_lp AS lp
  FROM enriched
  WHERE event_name = 'session_start'
),
cv_users AS (
  SELECT DISTINCT
    month,
    user_pseudo_id,
    session_medium AS medium,
    session_source AS source,
    session_lp AS lp
  FROM enriched
  WHERE cv = 1
)

-- 月×媒体×ソース×LP単位で集計
SELECT
  u.month,
  u.medium,
  u.source,
  u.lp,
  COUNT(DISTINCT u.user_pseudo_id) AS users,
  COUNT(DISTINCT c.user_pseudo_id) AS cv
FROM users AS u
LEFT JOIN cv_users AS c
  ON u.month = c.month
 AND u.user_pseudo_id = c.user_pseudo_id
 AND u.lp = c.lp
GROUP BY u.month, u.medium, u.source, u.lp
ORDER BY u.month, u.medium, u.source;
""", to_dataframe=True)
df_h['clinic'] = df_h['lp'].apply(infer_clinic_from_lp)

# 加工前の df_h はそのまま保持
df_proc = df_h.copy()

# channel列を追加
df_proc["channel"] = np.select(
    [
        df_proc["lp"].str.contains("plus.dentamap.jp", na=False),
        df_proc["lp"].str.contains(".kyousei-clinic.jp", na=False),
    ],
    [
        "dentamap",
        "kyousei-clinic.jp",
    ],
    default="(not set)"
)

# source 正規化
df_proc = apply_normalization(df_proc, "source", source_map)

# selected_sites の domain に一致する source を除外
exclude_domains = [s["domain"] for s in selected_sites if "domain" in s]
pattern = "|".join(map(re.escape, exclude_domains))
df_proc = df_proc[~df_proc["source"].str.contains(pattern, na=False)]

# clinic="不明" も除外
df_proc = df_proc[df_proc["clinic"] != "不明"]

# 列順を整理
col_order = ["month", "clinic", "channel", "medium", "source", "lp", "users", "cv"]
df_proc = df_proc[[c for c in col_order if c in df_proc.columns]]

# groupby & sum（最後の2列は整数型）
group_keys = col_order[:-2]
metric_cols = col_order[-2:]

df_proc = (
    df_proc.groupby(group_keys, dropna=False)[metric_cols]
    .sum(min_count=1)
    .reset_index()
)

# int型を保持
for col in metric_cols:
    df_proc[col] = df_proc[col].fillna(0).astype(int)

# Google Sheets保存
sheet_name = "_ch2-m"
mg.sheets_service.upsert_df(
    URL,
    sheet_name,
    df_proc,
    keys=group_keys,
    columns=col_order,
    sort_by=col_order,
    create_if_missing=True,
)


In [ ]:
#@title Search Consoleレポート更新 →_device, _query, _query_cat


def classify_page(df, mapping, new_col='page_category'):
    if df.empty or not mapping:
        return df

    def map_page(url):
        for pattern, label in mapping.items():
            try:
                if re.search(pattern, url):
                    return label
            except re.error as e:
                print(f"⚠️ Invalid pattern '{pattern}': {e}")
        return 'other'

    df[new_col] = df['page'].apply(map_page)
    return df

):
    """
    Fetch all Search Console rows using paging, retry, and safe parsing.
    Optionally apply cleaning and aggregation logic at the end.

    Args:
        start_date (str): YYYY-MM-DD
        end_date (str): YYYY-MM-DD
        site_url (str): e.g., 'https://example.com'
        dimensions (list): e.g., ['query', 'page'], ['page'], ['query', 'page', 'device']
        country (str or None): e.g., 'jpn', or None to disable filtering
        row_limit (int): Max rows per request (up to 25,000)
        verbose (bool): Print logs per page (default: False)
        max_rows (int): Safety limit to avoid runaway loops
        max_retries (int): Retries per failed call
        backoff_factor (float): Sleep multiplier for retries
        clean (bool):

    Returns:
        pd.DataFrame: Combined result with keys and metrics
    """
    creds = service_account.Credentials.from_service_account_file(CREDS_PATH)
    creds = creds.with_scopes(['https://www.googleapis.com/auth/webmasters.readonly'])
    webmasters = build('searchconsole', 'v1', credentials=creds)

    all_rows = []
    start_row = 0
    total_rows_fetched = 0

    while True:
        request_body = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': dimensions,
            'rowLimit': row_limit,
            'startRow': start_row
        }

        if country:
            request_body['dimensionFilterGroups'] = [{
                'filters': [{
                    'dimension': 'country',
                    'operator': 'equals',
                    'expression': country
                }]
            }]

        for attempt in range(max_retries):
            try:
                response = webmasters.searchanalytics().query(siteUrl=site_url, body=request_body).execute()
                rows = response.get('rows', [])
                break  # Success
            except HttpError as e:
                wait = backoff_factor * (2 ** attempt)
                if verbose:
                    print(f"⚠️ API error on startRow={start_row}, retrying in {wait:.1f}s... ({attempt + 1}/{max_retries})\n{e}")
                time.sleep(wait)
            except Exception as e:
                print(f"❌ Fatal error on startRow={start_row}: {e}")
                return pd.DataFrame()
        else:
            print(f"❌ Max retries exceeded at startRow={start_row}")
            break

        if not rows:
            break

        if verbose:
            print(f"📄 {site_url} - Fetched {len(rows)} rows (startRow={start_row})")

        all_rows.extend(rows)
        total_rows_fetched += len(rows)
        start_row += len(rows)

        if len(rows) < row_limit:
            break  # No more pages

        if total_rows_fetched >= max_rows:
            print(f"⚠️ Reached max_rows limit ({max_rows}), stopping.")
            break

    if not all_rows:
        if verbose:
            print("⚠️ No rows collected after paging.")
        return pd.DataFrame()

    # Convert to DataFrame
    data = []
    for i, r in enumerate(all_rows):
        keys = r.get("keys", [])
        if len(keys) < len(dimensions):
            if verbose:
                print(f"⚠️ Skipping row {i} due to missing keys: {keys}")
            continue
        try:
            row_data = {dim: keys[idx] for idx, dim in enumerate(dimensions)}
            row_data.update({
                'clicks': r.get('clicks', 0),
                'impressions': r.get('impressions', 0),
                'position': r.get('position', 0.0)
            })
            data.append(row_data)
        except Exception as e:
            print(f"❌ Error in row {i}: {e}")

    if not data:
        if verbose:
            print("⚠️ No usable rows after parsing.")
        return pd.DataFrame()

    df = pd.DataFrame(data)
    if verbose:
        print(f"✅ Total rows returned: {len(df):,}")

    # Clean if requested
    if clean:
        df['month'] = pd.to_datetime(start_date).strftime('%Y%m')
        return aggregate_search_console_data(df)

    return df

):
    """
    Generic GSC data fetcher.
    - Aggregates per specified dimensions (e.g., ['device'], ['query', 'page'])
    - Adds 'clinic' and 'month' if specified
    """
    dfs = []
    selected_sites = [s for s in sites if not clinic_filter or s["clinic"] in clinic_filter]

    for site in selected_sites:
        clinic = site['clinic']
        site_url = site.get('url', '').strip('/')
        print(f"🔍 Fetching GSC data: {clinic} ({site_url}) [{', '.join(dimensions)}]")

        df = query_search_console(
            start_date=start_date,
            end_date=end_date,
            site_url=site_url,
            dimensions=dimensions,
            country=country,
            clean=True
        )

        if not df.empty:
            if include_clinic:
                df['clinic'] = clinic
            if include_month:
                df['month'] = pd.to_datetime(start_date).strftime("%Y%m")
            dfs.append(df)

    if not dfs:
        print("⚠️ No GSC data retrieved.")
        return pd.DataFrame()

    df_all = pd.concat(dfs, ignore_index=True)

    if select_columns:
        df_all = df_all[[col for col in select_columns if col in df_all.columns]]

    return df_all

def apply_query_mapping(df, mapping):
    """
    Apply regex-based rule mapping to the 'query' column in place.
    """
    if df.empty or not mapping:
        return df

    def map_query(q):
        for pattern, mapped in mapping.items():
            try:
                if re.search(pattern, q):
                    return mapped
            except re.error as e:
                print(f"⚠️ Invalid regex pattern skipped: {pattern} → {e}")
        return q

    df['query'] = df['query'].astype(str).apply(map_query)
    return df

def deduplicate_queries(df):
    if df.empty:
        return df

    # Create a key by removing all whitespace
    df['query_key'] = df['query'].str.replace(r"\s+", "", regex=True)

    # Sort to prefer highest-impression query in each group
    df_sorted = df.sort_values(by=['month', 'clinic', 'page', 'query_key', 'impressions'], ascending=[True, True, True, True, False])

    # Pick top query (with max impressions) per group
    top_queries = df_sorted.groupby(['month', 'clinic', 'page', 'query_key'], as_index=False).first()

    # Aggregate metrics per group
    agg_metrics = df.groupby(['month', 'clinic', 'page', 'query_key']).agg(
        impressions=('impressions', 'sum'),
        clicks=('clicks', 'sum'),
        weighted_pos=('position', lambda x: (x * df.loc[x.index, 'impressions']).sum())
    ).reset_index()

    # Merge to get best-query string back
    result = pd.merge(agg_metrics, top_queries[['month', 'clinic', 'page', 'query_key', 'query']], on=['month', 'clinic', 'page', 'query_key'], how='left')

    # Finalize
    result['position'] = (result['weighted_pos'] / result['impressions']).round(6)
    result = result.drop(columns=['query_key', 'weighted_pos'])

    # Reorder columns
    return result[['month', 'clinic', 'query', 'page', 'impressions', 'clicks', 'position']]

def filter_by_clinic_thresholds(df, thresholds_df):
    if df.empty:
        return df

    df_filtered = pd.DataFrame()

    for _, row in thresholds_df.iterrows():
        clinic = row['clinic']
        min_imp = row['min_impressions']
        max_pos = row['max_position']

        df_sub = df[df['clinic'] == clinic]

        df_sub = df_sub[
            ~((df_sub['clicks'] == 0) & ((df_sub['impressions'] < min_imp) | (df_sub['position'] > max_pos)))
        ]

        df_filtered = pd.concat([df_filtered, df_sub], ignore_index=True)

    return df_filtered

def _normalize_gsc_keys(df):
    df = df.copy()
    df['month'] = df['month'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)
    df['clinic'] = df['clinic'].astype(str).str.strip()
    df['query'] = df['query'].astype(str).str.strip().str.lstrip("'")  # 🧼 remove saved quote
    df['page'] = df['page'].astype(str).str.strip().str.lower()

    def fix_query(val):
        if isinstance(val, float) and val.is_integer():
            return str(int(val))
        return str(val)
    df['query'] = df['query'].apply(fix_query)

    return df

def _force_text_on_numeric_queries(df):
    """
    Prevent Google Sheets from converting numeric queries (e.g. phone numbers)
    by prepending a single quote.
    """
    df = df.copy()
    df['query'] = df['query'].apply(
        lambda x: f"'{x}" if re.fullmatch(r"\d+", str(x)) else x
    )
    return df

# PageとQueryのデータ取得

# TARGET_MONTHS_AGO = 0 # 通常はコメントアウト

date_from = get_past_date(n_months=TARGET_MONTHS_AGO)
date_to = (get_past_date(n_months=TARGET_MONTHS_AGO - 1, return_date_obj=True) - timedelta(days=1)).isoformat()
print(f"期間：{date_from}〜{date_to}")

CLINIC_FILTER = [
    "札幌",
    "仙台",
    "池袋",
    "新宿",
    "渋谷",
    "東京",
    "横浜",
    "梅田",
    "難波",
    "博多",
    "天神",
    # "Gr.",
    # "dentamap",
]

# 1. Get Data, clean & decode pages, group by query + page
gsc_df = mg.gsc_service.fetch_sites(
    sites=sites,
    clinic_filter=CLINIC_FILTER,
    start_date=date_from,
    end_date=date_to,
    dimensions=['query', 'page'],
    country='jpn',
    include_clinic=True,
    include_month=True,
    clean=True,
    aggregate=True,
)

# 2. Apply regex-based query mapping
gsc_df = apply_query_mapping(gsc_df, query_map)

# 3. Group query variations by clinic + page + query_key (e.g., "abc", "a bc")
gsc_df = deduplicate_queries(gsc_df)

# 4. 分類と集計
gsc_df2 = classify_page(gsc_df, page_map)
summary_df = (
    gsc_df2
    .groupby(['month', 'clinic', 'page_category'], as_index=False)
    .agg(
        impressions=('impressions', 'sum'),
        clicks=('clicks', 'sum')
    )
    .sort_values(['month', 'clinic', 'impressions'], ascending=[True, True, False])
)

# 5. 差分のみ_query_catシートへ追記
mg.sheets_service.upsert_df(
    URL,
    "_query_cat",
    summary_df,
    keys=["month", "clinic", "page_category"],
    columns=["month", "clinic", "page_category", "impressions", "clicks"],
    sort_by=["month", "clinic", "page_category"],
    create_if_missing=True,
)

if TARGET_MONTHS_AGO == 1:
    # Filter low-value rows at the very end
    gsc_df_filtered = filter_by_clinic_thresholds(gsc_df, df_config)

    # Save
    gsc_df_filtered = _normalize_gsc_keys(gsc_df_filtered)
    gsc_df_filtered = _force_text_on_numeric_queries(gsc_df_filtered)
    mg.sheets_service.upsert_df(
        URL,
        "_query",
        gsc_df_filtered,
        keys=["month", "clinic", "query", "page"],
        columns=["month", "clinic", "query", "page", "impressions", "clicks", "position"],
        sort_by=["month", "clinic", "query", "page"],
        create_if_missing=True,
    )

# サイト全体の合計指標を月別に得る

# 1. APIからデータ取得
df = mg.gsc_service.fetch_sites(
    sites=sites,
    clinic_filter=CLINIC_FILTER,
    start_date=date_from,
    end_date=date_to,
    dimensions=['device'],
    country='jpn',
    include_clinic=True,
    include_month=True,
    clean=True,
    aggregate=True,
)
# 2. 差分のみシートへ追記
if df.empty:
    print("⚠️ No data to save to _device.")
else:
    print(f"📊 {len(df):,} rows prepared for saving to _device.")
    mg.sheets_service.upsert_df(
        URL,
        "_device",
        df,
        keys=["month", "clinic", "device"],
        columns=["month", "clinic", "device", "impressions", "clicks", "position"],
        sort_by=["month", "clinic", "device"],
        create_if_missing=True,
    )
